# Importaciones

In [135]:
import json 
import datetime
import re

import pandas as pd 
from googleapiclient.errors import HttpError
from Google import Create_Service

# Obtención de datos

## Llamamiento a la API de YouTube

Para obtener los datos vamos a usar las descripciones en cada video de las mañaneras. Pues nos provee de un resumen del video del día. De esta manera ya no tenemos que lidiar con conversaciones del presidente cuando usamos las transcripciones.

Función que utiliza expresiones regulares para obtener los segundos totales de una duración dada.

In [136]:
def convert_duration(duration):
    try:
        h = int(re.search('\d+H', duration)[0][:-1]) * 60**2  if re.search('\d+H', duration) else 0 # hour
        m = int(re.search('\d+M', duration)[0][:-1]) * 60  if re.search('\d+M', duration) else 0 # minute
        s = int(re.search('\d+S', duration)[0][:-1])  if re.search('\d+S', duration) else 0 # second
        return h + m + s
    except Exception as e:
        print(e)
        return 0 

In [137]:
def retrieve_playlists(service, channelId):
    playlists =[]
    try:
        response = service.playlists().list(
            part = 'contentDetails, snippet, status',
            channelId = channelId,
            maxResults = 50
        ).execute()

        playlists.extend(response.get('items'))
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = service.playlists().list(
                part = 'contentDetails, snippet, status',
                channelId = channelId,
                maxResults = 50,
                pageToken = nextPageToken
            ).execute()

            playlists.extend(response.get('items'))
            nextPageToken = response.get('nextPageToken')

        return playlists

    except HttpError as e:
        errMsg = json.loads(e.content)
        print("HTTP Error: ")
        print(errMsg['error']['message'])
        return []

Función que hace la petición a la API de youtube

In [138]:
def retrieve_playlists_items(service, playlistId):
    items =[]
    try:
        response = service.playlistItems().list(
            part = 'contentDetails, snippet, status',
            playlistId = playlistId,
            maxResults = 5
        ).execute()

        items.extend(response.get('items'))
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = service.playlistItems().list(
                part = 'contentDetails, snippet, status',
                playlistId = playlistId,
                maxResults = 5,
                pageToken = nextPageToken
            ).execute()

            items.extend(response.get('items'))
            nextPageToken = response.get('nextPageToken')

        return items

    except HttpError as e:
        errMsg = json.loads(e.content)
        print("HTTP Error: ")
        print(errMsg['error']['message'])
        return []

Creando constantes para llamar a la API

In [139]:
CLIENT_SECRETS_FILE = 'client_secret.json'  # Archivo .json que se obtiene al crear las credenciales OAuth en https://console.cloud.google.com/
API_NAME = 'youtube'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']

service = Create_Service(CLIENT_SECRETS_FILE, API_NAME, API_VERSION, SCOPES)

client_secret.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.readonly'],)
['https://www.googleapis.com/auth/youtube.readonly']
youtube service created successfully


El objeto <code>service</code> nos da accesos a todos los recursos de la API

In [140]:
dir(service)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_basic_methods', '_add_nested_resources', '_add_next_methods', '_baseUrl', '_developerKey', '_dynamic_attrs', '_http', '_model', '_requestBuilder', '_resourceDesc', '_rootDesc', '_schema', '_set_dynamic_attr', '_set_service_methods', 'abuseReports', 'activities', 'captions', 'channelBanners', 'channelSections', 'channels', 'close', 'commentThreads', 'comments', 'i18nLanguages', 'i18nRegions', 'liveBroadcasts', 'liveChatBans', 'liveChatMessages', 'liveChatModerators', 'liveStreams', 'members', 'membershipsLevels', 'new_batch_http_request', 'playlistItems', 'playlists', 'search', 'subscri

Hacemos una llamada a a función de <code>retrieve_playlists_items</code> para obtener todos los videos una playlist especifica 

In [141]:
playlistId = 'PLRnlRGar-_296KTsVL0R6MEbpwJzD8ppA' # id de la playlist de las mañaneras
playlists_items = retrieve_playlists_items(service, playlistId)

In [142]:
playlists_items[0] # El último video agregado a la playlist

{'kind': 'youtube#playlistItem', 'etag': '8WJEXuec_liHMLQ3MWFIqoEQmWs', 'id': 'UExSbmxSR2FyLV8yOTZLVHNWTDBSNk1FYnB3SnpEOHBwQS4zRDNFM0Q1MzY2NTYwMTJC', 'snippet': {'publishedAt': '2023-04-24T12:44:41Z', 'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg', 'title': 'Conferencia de prensa matutina desde Palacio Nacional. Lunes 24 de abril 2023 | Presidente AMLO', 'description': 'Conferencia de prensa matutina, desde Palacio Nacional. Lunes 24 de abril 2023 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n15:15 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n16:48 Comparativo de marcas y precios de combustibles\n25:07 Comparativo de marcas y precios de productos de primera necesidad\n28:30 Reporte de construcción del Tren Maya en tramo Escárcega-Calkiní\n32:57 Informe del consorcio Carso FCC en el Tramo 2 del Tren Maya\n34:14 Salvamento arqueológico en el proyecto del Tren Maya\n40:26 Informe de la Semarnat en el Tramo 2 del Tren Maya\n42:33 Informe de la Secretaría del B

## Guardando los datos en un DataFrame

Podemos obtener cada elemento de la variable utilizando las posiciones.

In [152]:
playlists_items[0]['snippet']

{'publishedAt': '2023-04-24T12:44:41Z', 'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg', 'title': 'Conferencia de prensa matutina desde Palacio Nacional. Lunes 24 de abril 2023 | Presidente AMLO', 'description': 'Conferencia de prensa matutina, desde Palacio Nacional. Lunes 24 de abril 2023 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n15:15 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n16:48 Comparativo de marcas y precios de combustibles\n25:07 Comparativo de marcas y precios de productos de primera necesidad\n28:30 Reporte de construcción del Tren Maya en tramo Escárcega-Calkiní\n32:57 Informe del consorcio Carso FCC en el Tramo 2 del Tren Maya\n34:14 Salvamento arqueológico en el proyecto del Tren Maya\n40:26 Informe de la Semarnat en el Tramo 2 del Tren Maya\n42:33 Informe de la Secretaría del Bienestar en el Tramo 2 del Tren Maya\n42:51 Informe de los Programas para el Bienestar\n1:10:05 Sesión de preguntas y respuestas\n\nSigue las actividades del pre

Hay que crear un dataframe con la información necesaria para hacer el entrenamiento. Estos datos son: EL título, la descrición y la fecha de publicación

In [150]:

data = []

for item in playlists_items:
    snippet = item['snippet']
    published_at = snippet['publishedAt']
    title = snippet['title']
    description = snippet['description']
    
    data.append([published_at, title, description])
    
df = pd.DataFrame(data, columns=['published_at', 'title', 'description'])
df = df.head(1085)

print(df)


              published_at                                              title   
0     2023-04-24T12:44:41Z  Conferencia de prensa matutina desde Palacio N...  \
1     2023-04-21T05:16:34Z  Seguridad y bienestar mejoran en Veracruz. Con...   
2     2023-04-20T11:50:16Z  Recursos de venta de avión serán para construi...   
3     2023-04-19T12:49:04Z  Postura ante resolución de la SCJN sobre Guard...   
4     2023-04-18T12:45:45Z  Padres y madres pueden ayudar a prevenir adicc...   
...                    ...                                                ...   
1080  2019-03-08T21:59:25Z  Conferencia matutina: Avances del Gabinete de ...   
1081  2019-03-08T21:59:25Z  Hoy anunciamos la terna de candidatos a minist...   
1082  2019-03-08T21:59:25Z  Tercera conferencia de prensa matutina desde P...   
1083  2019-03-08T21:59:25Z     Diálogo con periodistas desde Palacio Nacional   
1084  2019-03-08T21:59:25Z  Primera conferencia de prensa matutina desde P...   

                           

## Una primera limpieza

In [156]:
df['description'][0]

'Conferencia de prensa matutina, desde Palacio Nacional. Lunes 24 de abril 2023 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n15:15 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n16:48 Comparativo de marcas y precios de combustibles\n25:07 Comparativo de marcas y precios de productos de primera necesidad\n28:30 Reporte de construcción del Tren Maya en tramo Escárcega-Calkiní\n32:57 Informe del consorcio Carso FCC en el Tramo 2 del Tren Maya\n34:14 Salvamento arqueológico en el proyecto del Tren Maya\n40:26 Informe de la Semarnat en el Tramo 2 del Tren Maya\n42:33 Informe de la Secretaría del Bienestar en el Tramo 2 del Tren Maya\n42:51 Informe de los Programas para el Bienestar\n1:10:05 Sesión de preguntas y respuestas\n\nSigue las actividades del presidente de México:\n\nSitio web:\nhttp://presidente.gob.mx\n\nYouTube:\nhttps://www.youtube.com/lopezobrador\n\nFacebook:\nhttps://facebook.com/lopezobrador.org.mx\n\nTwitter:\nhttps://twitter.com/lopezob

In [158]:
def replace_newlines(string):
    pattern = r'\n'
    replace_with = ' '
    return re.sub(pattern, replace_with, string)

In [165]:
def replace_minutes(string):
    pattern = r'[0-9]?[0-9]:[0-9][0-9]'
    replace_with = ''
    return re.sub(pattern, replace_with, string)

In [173]:
def remove_links(string):
    pattern = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    replace_with = ''
    return re.sub(pattern, replace_with, string)

In [183]:
def remove_extra_words(string):
    pattern = r'Sigue las actividades del presidente de México:  Sitio web:   YouTube:   Facebook:   Twitter:   Instagram:   Telegram:    Spotify: '
    replace_with = ''
    return re.sub(pattern, replace_with, string)

In [185]:
df['description'] = df['description'].apply(replace_newlines).apply(replace_minutes).apply(remove_links).apply(remove_extra_words)


In [187]:
df['description'][50]

'Conferencia de prensa matutina, desde Palacio Nacional. Martes 07 de febrero 2023 | Presidente AMLO.   Inicio de transmisión  Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador  Informe de seguridad  Informe Cero Impunidad 1: Informe sobre regularización de vehículos de procedencia extranjera 1: Informe sobre preliberaciones y amnistías 1: Informe del Tianguis del Bienestar 1: Informe sobre las capacidades del Aeropuerto Internacional Felipe Ángeles 1: Informe caso Genaro García Luna 1: Sesión de preguntas y respuestas  Informamos sobre las capacidades del Aeropuerto Internacional Felipe Ángeles en cuanto al transporte de carga y pasajeros. Es el más moderno, equipado y seguro del sistema aeroportuario del país.   El AIFA puede recibir hasta 443 mil toneladas al año, a diferencia del AICM, que en 2022 registró la llegada de un promedio de 250 mil toneladas en aviones dedicados a la carga internacional.  De conformidad con su programa maestro, el Aeropuerto In

In [178]:
df

,published_at,title,description
0,2023-04-24T12:44:41Z,Conferencia de prensa matutina desde Palacio N...,"Conferencia de prensa matutina, desde Palacio ..."
1,2023-04-21T05:16:34Z,Seguridad y bienestar mejoran en Veracruz. Con...,"Conferencia de prensa matutina, desde Palacio ..."
2,2023-04-20T11:50:16Z,Recursos de venta de avión serán para construi...,"Conferencia de prensa matutina, desde Palacio ..."
3,2023-04-19T12:49:04Z,Postura ante resolución de la SCJN sobre Guard...,"Conferencia de prensa matutina, desde Palacio ..."
4,2023-04-18T12:45:45Z,Padres y madres pueden ayudar a prevenir adicc...,"Conferencia de prensa matutina, desde Palacio ..."
...,...,...,...
1080,2019-03-08T21:59:25Z,Conferencia matutina: Avances del Gabinete de ...,"Conferencia de prensa matutina, desde Palacio ..."
1081,2019-03-08T21:59:25Z,Hoy anunciamos la terna de candidatos a minist...,Cuarta conferencia de prensa matutina desde el...
1082,2019-03-08T21:59:25Z,Tercera conferencia de prensa matutina desde P...,Tercera conferencia de prensa matutina desde e...
1083,2019-03-08T21:59:25Z,Diálogo con periodistas desde Palacio Nacional,Segunda conferencia de prensa matutina del pre...
